# Final assignment week3. by Davit Harutyunyan

## Par I: task description. 
    1. Start by creating a new Notebook for this assignment.
    
    2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas  dataframe like the one shown below:
    3. To create the above dataframe:

    The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
    Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

    If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
    Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

4. Submit a link to your Notebook on your Github repository.(10 marks)

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library
import numpy as np # library to handle data in a vectorized manner
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Using Webscraping to Extract table from the wiki site

In [2]:
html_data=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
html_data
print('wiki page is successfully read out')

wiki page is successfully read out


 Parse the html data using beautiful_soup.

In [3]:
beautiful_soup= BeautifulSoup(html_data.content, 'html.parser')

In [4]:
beautiful_soup.title

<title>List of postal codes of Canada: M - Wikipedia</title>

In [5]:
# read table with  beautiful_soup and then convert data to dataframe.
table = beautiful_soup.find_all('table')[0] 
#table
Postal_code=[]
Borough=[]
Neighborhood=[]
for child in beautiful_soup.find_all('table')[0].children:
    for td in child:
        try:
            #print('--------------------')
            #print(td.text)            
            qaq=[]
            for line in td.text.splitlines():
                line
                qaq.append(line)
            Postal_code.append(qaq[1])
            Borough.append(qaq[3])
            Neighborhood.append(qaq[5])            
        except:            
            continue
Toronto_data=pd.DataFrame([Postal_code[1:],Borough[1:],Neighborhood[1:]]).T
Toronto_data.columns=['Postal_code','Borough','Neighborhood']

In [6]:
Toronto_data.head()

,Postal_code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
Toronto_data=Toronto_data[Toronto_data.Borough!='Not assigned']
Toronto_data=Toronto_data.reset_index(drop=True)
Toronto_data.head()


,Postal_code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

In [8]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
t=Toronto_data.Neighborhood=='Not assigned'
indexx=[i for i, x in enumerate(t) if x]
Toronto_data.loc[indexx,'Neighborhood']=Toronto_data.loc[indexx,'Borough']

In [9]:
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
Toronto_data.shape

(103, 3)

## Par II: task description. 

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking  postal code M5G as an example, your code would look something like this:

In [10]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
postal_code='M5K'
# loop until you get the coordinates
s=0;
while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))    
    lat_lng_coords = g.latlng
    s=s+1
    if s>10:
        print('----------- this code does not work!!!!!!!!!!!!!!')
        break
        
if g.ok:
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]

----------- this code does not work!!!!!!!!!!!!!!


In [11]:
# lets load the csv file
coord=pd.read_csv('Geospatial_Coordinates.csv')
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
# now find respective poscodes and add coordinates to map them correctly to the dataframe
a=Toronto_data['Postal_code']
b=coord['Postal Code']
both = set(a).intersection(b)

a_list=list(a)
b_list=list(b)

indices_a = [a_list.index(x) for x in both]
indices_b = [b_list.index(x) for x in both]
# now we have respective indices, hence we can merge the two data frames
# first let us reorder both dataframes
Toronto_data=Toronto_data.iloc[indices_a] 
coord=coord.iloc[indices_b]
#print(Toronto_data.head())
#print(coord.head())
# and now join then
Toronto_data=Toronto_data.join(coord[['Latitude','Longitude']])

In [13]:
Toronto_data.head()

,Postal_code,Borough,Neighborhood,Latitude,Longitude
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
0,M3A,North York,Parkwoods,43.806686,-79.194353
91,M4W,Downtown Toronto,Rosedale,43.636258,-79.498509
14,M4C,East York,Woodbine Heights,43.815252,-79.284577
17,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.803762,-79.363452


## part III: task description

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. 

Just make sure:

    to add enough Markdown cells to explain what you decided to do and to report any observations you make. 
    to generate maps to visualize your neighborhoods and how they cluster together. 

In [14]:
Toronto_data['Borough'].unique()

array(['Downtown Toronto', 'North York', 'East York', 'Etobicoke', 'York',
       'Scarborough', 'West Toronto', 'Central Toronto', 'Mississauga',
       'East Toronto'], dtype=object)

In [15]:
Toronto_data.shape
neighborhoods=Toronto_data.copy(deep=True)

In [16]:
neighborhoods.head()

,Postal_code,Borough,Neighborhood,Latitude,Longitude
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
0,M3A,North York,Parkwoods,43.806686,-79.194353
91,M4W,Downtown Toronto,Rosedale,43.636258,-79.498509
14,M4C,East York,Woodbine Heights,43.815252,-79.284577
17,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.803762,-79.363452


In [17]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.6534817, -79.3839347.


In [18]:
# create map of Toronot using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [19]:
CLIENT_ID = 'VQS1IZPHL4UH3NCMLETLTV1QZTDWWBKRTDFQL101HW3ZND2C' # your Foursquare ID
CLIENT_SECRET = 'EA5IKNG320UTBH5HGYC2ST3IEQWWQFJCLRO11CHM0NOAXH5U' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VQS1IZPHL4UH3NCMLETLTV1QZTDWWBKRTDFQL101HW3ZND2C
CLIENT_SECRET:EA5IKNG320UTBH5HGYC2ST3IEQWWQFJCLRO11CHM0NOAXH5U


In [20]:
explore_this_Borough_keyword='Toronto'
this_df_explore=Toronto_data[Toronto_data['Borough'].str.contains(explore_this_Borough_keyword)]


In [21]:
# plot this chosen Borough neighborhoods
# create map of Manhattan using latitude and longitude values
new_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(this_df_explore['Latitude'], this_df_explore['Longitude'], this_df_explore['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(new_map)  
    
new_map

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
#  get venues
this_venues = getNearbyVenues(names=this_df_explore['Neighborhood'],
                                   latitudes=this_df_explore['Latitude'],
                                   longitudes=this_df_explore['Longitude']
                                  )

Garden District, Ryerson
Rosedale
Parkdale, Roncesvalles
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Roselawn
Stn A PO Boxes
Davisville North
Regent Park, Harbourfront
First Canadian Place, Underground city
St. James Town, Cabbagetown
Lawrence Park
Christie
Berczy Park
St. James Town
Richmond, Adelaide, King
Kensington Market, Chinatown, Grange Park
Brockton, Parkdale Village, Exhibition Place
The Beaches
Little Portugal, Trinity
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Queen's Park, Ontario Provincial Government
Studio District
University of Toronto, Harbord
Central Bay Street
Church and Wellesley
Forest Hill North & West, Forest Hill Road Park
North Toronto West,  Lawrence Park
Toronto Dominion Centre, Design Exchange
Moore Park, Summerhill East
Commerce Court, Victoria Hotel
High Park, The Junction South
Davisville
Runnymede, Swansea
The Danforth West, Riverdale
Harbourfront East, Union Station, 

In [24]:
print('There are {} uniques categories.'.format(len(this_venues['Venue Category'].unique())))

There are 198 uniques categories.


In [25]:
#analyze each neighborhood
# one hot encoding
this_onehot = pd.get_dummies(this_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
this_onehot['Neighborhood'] = this_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [this_onehot.columns[-1]] + list(this_onehot.columns[:-1])
this_onehot = this_onehot[fixed_columns]


In [26]:
# group
this_grouped = this_onehot.groupby('Neighborhood').mean().reset_index()


In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = this_grouped['Neighborhood']

for ind in np.arange(this_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(this_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Gastropub,Brewery,Café,American Restaurant,Bakery,Yoga Studio,Middle Eastern Restaurant,Pet Store,Park
1,"Business reply mail Processing Centre, South C...",Bus Line,Sandwich Place,Pizza Place,Mobile Phone Shop,Women's Store,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
2,"CN Tower, King and Spadina, Railway Lands, Har...",Yoga Studio,Auto Workshop,Restaurant,Brewery,Skate Park,Light Rail Station,Smoke Shop,Burrito Place,Farmers Market,Fast Food Restaurant
3,Central Bay Street,Coffee Shop,Grocery Store,Pharmacy,Pizza Place,Butcher,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner
4,Christie,Park,Construction & Landscaping,Food & Drink Shop,Women's Store,Curling Ice,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [29]:
# Run k-means to cluster the neighborhood into 5 clusters
# set number of clusters
kclusters = 5

this_grouped_clustering = this_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(this_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 3, 1, 1, 3, 1, 1])

In [30]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

this_merged = this_df_explore

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
this_merged = this_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

this_merged.head() # check the last columns!

,Postal_code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848,1.0,General Entertainment,Café,Skating Rink,College Stadium,Cupcake Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
91,M4W,Downtown Toronto,Rosedale,43.636258,-79.498509,2.0,Baseball Field,Women's Store,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
75,M6R,West Toronto,"Parkdale, Roncesvalles",43.669542,-79.422564,1.0,Grocery Store,Café,Park,Candy Store,Coffee Shop,Nightclub,Italian Restaurant,Baby Store,Athletics & Sports,Restaurant
86,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.636966,-79.615819,1.0,Coffee Shop,Hotel,Gym,Sandwich Place,Intersection,Burrito Place,Mediterranean Restaurant,Food Truck,Fried Chicken Joint,Middle Eastern Restaurant
62,M5N,Central Toronto,Roselawn,43.733283,-79.419750,1.0,Italian Restaurant,Sushi Restaurant,Coffee Shop,Sandwich Place,Juice Bar,Indian Restaurant,Pub,Pizza Place,Fast Food Restaurant,Pharmacy


In [31]:
# Finally, let's visualize the resulting clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(this_merged['Latitude'], this_merged['Longitude'], this_merged['Neighborhood'], this_merged['Cluster Labels']):
    if np.isnan(cluster)==False:
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)        
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[int(cluster)-1],
            fill_opacity=0.7).add_to(map_clusters)

map_clusters